In [ ]:
import pandas as pd
import numpy as np

In [ ]:
census_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Census_Income/Training_set_census.csv" )

In [ ]:
census_data.head()

,age,class_of_worker,industry_code,occupation_code,education,wage_per_hour,enrolled_in_edu_inst_lastwk,marital_status,major_industry_code,major_occupation_code,race,hispanic_origin,sex,member_of_labor_union,reason_for_unemployment,full_parttime_employment_stat,capital_gains,capital_losses,dividend_from_Stocks,tax_filer_status,region_of_previous_residence,state_of_previous_residence,d_household_family_stat,d_household_summary,migration_msa,migration_reg,migration_within_reg,live_1_year_ago,migration_sunbelt,num_person_Worked_employer,family_members_under_18,country_father,country_mother,country_self,citizenship,business_or_self_employed,fill_questionnaire_veteran_admin,veterans_benefits,weeks_worked_in_year,year,income_level
0,23,Private,43,22,Some college but no degree,0,College or university,Never married,Education,Adm support including clerical,White,All other,Male,Not in universe,Not in universe,Full-time schedules,0,0,0,Single,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,NaN,NaN,NaN,Not in universe under 1 year old,NaN,4,Not in universe,Peru,Peru,United-States,Native- Born in the United States,0,Not in universe,2,30,95,0
1,24,Private,34,2,Bachelors degree(BA AB BS),0,Not in universe,Never married,Finance insurance and real estate,Executive admin and managerial,White,All other,Male,No,Not in universe,Children or Armed Forces,0,0,0,Single,West,California,Nonfamily householder,Householder,MSA to MSA,Different county same state,Different county same state,No,No,4,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,26,94,0
2,38,Private,34,2,Masters degree(MA MS MEng MEd MSW MBA),0,Not in universe,Married-civilian spouse present,Finance insurance and real estate,Executive admin and managerial,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,250,Joint both under 65,Not in universe,Not in universe,Householder,Householder,Nonmover,Nonmover,Nonmover,Yes,Not in universe,4,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,1
3,33,Not in universe,0,0,High school graduate,0,Not in universe,Married-civilian spouse present,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Joint both under 65,Not in universe,Not in universe,Child 18+ ever marr RP of subfamily,Child 18 or older,NaN,NaN,NaN,Not in universe under 1 year old,NaN,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,0
4,13,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,NaN,NaN,NaN,Not in universe under 1 year old,NaN,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,95,0


In [ ]:
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,12 mins 24 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.7
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_m438e9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.157 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
data = h2o.import_file("https://raw.githubusercontent.com/dphi-official/Datasets/master/Census_Income/Training_set_census.csv")


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
val = h2o.import_file('https://raw.githubusercontent.com/dphi-official/Datasets/master/Census_Income/Testing_set_census.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
train,test = data.split_frame([0.8])

In [ ]:
features = train.columns
features.remove('income_level')
y='income_level'
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
aml = H2OAutoML(max_models=10,seed=1,nfolds=5,verbosity='warn')
aml.train(x=features,y='income_level',training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
aml.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20200809_180358,0.955171,0.138421,0.71852,0.188905,0.19363,0.0374925
StackedEnsemble_BestOfFamily_AutoML_20200809_180358,0.954484,0.138888,0.716156,0.18031,0.194006,0.0376385
XGBoost_3_AutoML_20200809_180358,0.954316,0.124639,0.715585,0.185268,0.19032,0.0362216
GBM_2_AutoML_20200809_180358,0.953411,0.126412,0.708495,0.191955,0.191571,0.0366994
GBM_3_AutoML_20200809_180358,0.953106,0.126856,0.707282,0.195183,0.191921,0.0368338
XGBoost_1_AutoML_20200809_180358,0.953089,0.126717,0.705992,0.183313,0.192046,0.0368816
GBM_1_AutoML_20200809_180358,0.952961,0.127371,0.701859,0.186859,0.19215,0.0369217
GBM_5_AutoML_20200809_180358,0.952811,0.127247,0.705507,0.189814,0.192307,0.0369819
GBM_4_AutoML_20200809_180358,0.952235,0.128234,0.70353,0.190639,0.192933,0.0372231
XGBoost_2_AutoML_20200809_180358,0.951681,0.129146,0.696949,0.187301,0.193774,0.0375482


In [ ]:
y_pred = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [ ]:
y_pred

predict,p0,p1
0,0.982791,0.017209
0,0.982955,0.017045
0,0.983116,0.0168837
0,0.97834,0.0216598
0,0.977475,0.0225254
0,0.982431,0.0175693
0,0.983115,0.0168845
0,0.983053,0.0169475
0,0.970763,0.0292373
0,0.982451,0.0175488


In [ ]:
from sklearn.metrics import f1_score

In [ ]:
print(f1_score(test.as_data_frame(use_pandas=True)['income_level'],y_pred.as_data_frame(use_pandas=True)['predict']))

0.6411483253588518


In [ ]:
submission_pred = aml.predict(val)

stackedensemble prediction progress: |████████████████████████████████████| 100%


/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'd_household_family_stat' has levels not trained on: [Grandchild <18 ever marr not in subfamily]
  warnings.warn(w)


AttributeError: ignored

In [ ]:
submit = pd.DataFrame(submission_pred.as_data_frame(use_pandas=True)['predict'].tolist())

In [ ]:
from google.colab import files
submit.to_csv('prediction_results_HP1.csv')         
files.download('prediction_results_HP1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>